In [104]:
from database_handler import execute_query,create_connection,return_data_as_df
from lookups import InputTypes
db_session = create_connection()
query = """ SELECT * FROM dw_reporting.agg_daily_stock_prices_vs_sentiments
WHERE ticker = 'AAPL' """

df = return_data_as_df(query,InputTypes.SQL,db_session)
df

c:\Users\user\OneDrive\Documents\SE_Factory\FSD\Final_Project\usa_recession_analysis\database_handler.py:64: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_dataframe = pd.read_sql_query(con= db_session, sql= file_executor)


,date_ticker,date,ticker,stock_price,neg,neu,pos,compound
0,2023-10-06-AAPL,2023-10-06,AAPL,175.58,0.05,0.79,0.16,0.67
1,2023-10-05-AAPL,2023-10-05,AAPL,174.06,0.07,0.80,0.13,0.26
2,2023-10-04-AAPL,2023-10-04,AAPL,172.59,0.06,0.82,0.12,0.51
3,2023-10-03-AAPL,2023-10-03,AAPL,172.23,0.04,0.85,0.11,0.63


In [105]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from sklearn.model_selection import train_test_split

# Load your dataset
data = df.copy()

# Extract relevant features
X = data[['stock_price', 'compound']].values
y = data['stock_price'].values

# Normalize the data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Create sequences for the GRU model
sequence_length = 1  # Adjust as needed
X_sequences, y_sequences = [], []

for i in range(len(X) - sequence_length):
    X_sequences.append(X[i:i+sequence_length])
    y_sequences.append(y[i+sequence_length])

X_sequences, y_sequences = np.array(X_sequences), np.array(y_sequences)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)




In [136]:
X_train

array([[[0.54626866, 0.        ]],

       [[0.10746269, 0.6097561 ]]])

In [107]:
# Build the GRU model
model = Sequential()
model.add(GRU(units=50, activation='relu', input_shape=(sequence_length, X_train.shape[2])))
model.add(Dense(units=1))  # Output layer with one neuron for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Mean Squared Error on Test Data: {loss}')

# Make predictions
predictions = model.predict(X_test)



Epoch 1/50
1/1 [==============================] - 1s 860ms/step - loss: 29727.8477
Epoch 2/50
1/1 [==============================] - 0s 4ms/step - loss: 29724.9785
Epoch 3/50
1/1 [==============================] - 0s 4ms/step - loss: 29722.1523
Epoch 4/50
1/1 [==============================] - 0s 5ms/step - loss: 29719.3320
Epoch 5/50
1/1 [==============================] - 0s 5ms/step - loss: 29716.5039
Epoch 6/50
1/1 [==============================] - 0s 4ms/step - loss: 29713.6758
Epoch 7/50
1/1 [==============================] - 0s 4ms/step - loss: 29710.8398
Epoch 8/50
1/1 [==============================] - 0s 5ms/step - loss: 29707.9961
Epoch 9/50
1/1 [==============================] - 0s 5ms/step - loss: 29705.1211
Epoch 10/50
1/1 [==============================] - 0s 4ms/step - loss: 29702.2227
Epoch 11/50
1/1 [==============================] - 0s 3ms/step - loss: 29699.3125
Epoch 12/50
1/1 [==============================] - 0s 6ms/step - loss: 29696.3906
Epoch 13/50
1/1 [======

In [108]:
X

array([[1.        , 1.        ],
       [0.54626866, 0.        ],
       [0.10746269, 0.6097561 ],
       [0.        , 0.90243902]])

In [109]:
predictions

array([[0.8394536]], dtype=float32)

In [110]:
X_test

array([[[1., 1.]]])

In [87]:
X_test[:, -1, 0].reshape(-1, 1)

array([[1.]])

In [111]:
np.concatenate((X_test[:, -1, 0].reshape(-1, 1), predictions), axis=1)

array([[1.        , 0.83945358]])

In [115]:
scaler.inverse_transform(np.concatenate((X_test[:, -1, 0].reshape(-1, 1), predictions), axis=1))[:,1]

array([0.60417597])

In [116]:

# If needed, inverse transform the predictions to get them back to the original scale
predictions = scaler.inverse_transform(np.concatenate((X_test[:, -1, 0].reshape(-1, 1), predictions), axis=1))[:, 1]



In [123]:
predictions

array([0.60417597])

In [131]:
y_test.reshape(-1, 1)

array([[174.06]])

In [144]:
X_test

array([[[1., 1.]]])

In [138]:
scaler.inverse_transform(y_test.reshape(-1, 1))

ValueError: non-broadcastable output operand with shape (1,1) doesn't match the broadcast shape (1,2)

In [148]:
comparison = pd.DataFrame({'Predicted': predictions[-1]}, index=[1])
print(comparison)

   Predicted
1   0.604176


In [121]:
comparison = pd.DataFrame({'Actual': scaler.inverse_transform(y_test.reshape(-1, 1)).flatten(), 'Predicted': predictions[-1]})
print(comparison)

ValueError: non-broadcastable output operand with shape (1,1) doesn't match the broadcast shape (1,2)

In [118]:

# Compare predictions with actual values
comparison = pd.DataFrame({'Actual': scaler.inverse_transform(y_test.reshape(-1, 1)).flatten(), 'Predicted': predictions})
print(comparison)


ValueError: non-broadcastable output operand with shape (1,1) doesn't match the broadcast shape (1,2)